In [ ]:
#       Overall it appears students across all schools have a better passing rate in reading than maths
#       A higher budget doesn't appear to have any effect on passing averages
#       Scores by grade seem pretty flat by school, with the higher scores in reading than maths
#       Surprisingly, it appears the monies spent under $615 per student result in a greater passing average.
#       Small to Mid size schools, up tp 2,000 students, appear to perform better than larger schools.
#       Charter schools surpass district schools in overall passing rate by more than 30%

In [1]:
import pandas as pd
import numpy as np

school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
total_schools = school_data['school_name'].count()
total_students = student_data['student_name'].count()
total_budget = school_data['budget'].sum()
budget_per_student = school_data_complete.set_index('school_name')['budget']/school_data_complete.set_index('school_name')['size']
average_math_score = student_data['math_score'].sum() / total_students
average_reading_score = student_data['reading_score'].sum() / total_students

passing_students_math = student_data.loc[student_data['math_score'] > 70, :]['student_name'].count()
passing_rate_math = (passing_students_math / total_students)

passing_students_reading = student_data.loc[student_data['reading_score'] > 70, :]['student_name'].count()
passing_rate_reading = (passing_students_reading / total_students)

overall_passing_rate = (passing_rate_math + passing_rate_reading) / 2

district_summary_df = pd.DataFrame(
        {
            'Total Schools' : total_schools,
            'Total Students' : total_students,
            'Total Budget' : total_budget,
            'Average Math Score' : average_math_score,
            'Average Reading Score' : average_reading_score,
            '% Passing Math' : passing_rate_math,
            '% Passing Reading' : passing_rate_reading,
            '% Overall Passing Rate' : overall_passing_rate
        },
    index = [0]
)

district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,.2f}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('$ {:,.2f}'.format)
district_summary_df['Average Math Score'] = district_summary_df['Average Math Score'].map('{:,.2f}'.format)
district_summary_df['Average Reading Score'] = district_summary_df['Average Reading Score'].map('{:,.2f}'.format)
district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].map('{:,.2%}'.format)
district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].map('{:,.2%}'.format)
district_summary_df['% Overall Passing Rate'] = district_summary_df['% Overall Passing Rate'].map('{:,.2%}'.format)


district_summary_df = district_summary_df[['Total Schools', 'Total Students', 'Total Budget', 
                                           'Average Math Score', 'Average Reading Score', 
                                           '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170.00","$ 24,649,428.00",78.99,81.88,72.39%,82.97%,77.68%


In [3]:
group_school = school_data_complete.groupby(['school_name'])

s_type = group_school['type'].unique()
tot_students = group_school['Student ID'].nunique()
tot_budget = group_school['budget'].unique()
per_sb = tot_budget / tot_students
avg_math_school = group_school['math_score'].mean()
avg_reading_school = group_school['reading_score'].mean()

math_reading_df = pd.DataFrame(data=school_data_complete,columns=['student_name','school_name',
                                                                   'math_score','reading_score'])
math_students_filtered = math_reading_df[math_reading_df['math_score'] > 70] \
    .drop(columns=['student_name','reading_score'])
passing_math = math_students_filtered.groupby(['school_name'])
passing_math = passing_math.count()

tot_students = pd.DataFrame(data=tot_students)
result_df = pd.DataFrame()
result_df['% Passing Math'] = passing_math['math_score'] / tot_students['Student ID']

reading_students_filtered = math_reading_df[math_reading_df['reading_score'] > 70] \
    .drop(columns=['student_name','math_score'])
passing_reading = reading_students_filtered.groupby(['school_name'])
passing_reading = passing_reading.count()
result_df['% Passing Reading'] = passing_reading['reading_score'] / tot_students['Student ID']
result_df

result_df['% Overall Passing Rate'] = (result_df['% Passing Math'] + result_df['% Passing Reading']) / 2
result_df

summary_school_df = pd.DataFrame(data=s_type)

summary_school_df = summary_school_df.join([tot_students,tot_budget,per_sb,avg_math_school,avg_reading_school,
                        result_df['% Passing Math'],result_df['% Passing Reading'], 
                        result_df['% Overall Passing Rate']])

summary_school_df = summary_school_df.rename(columns={'school_name':'','Student ID': 'Total Students',
                                                     0: 'Per Student Budget',
                                                     'math_score': 'Average Math Score',
                                                     'reading_score': 'Average Reading Score','budget':'Total School Budget','type':'School Type'})

summary_school_df = summary_school_df.sort_values("% Overall Passing Rate",ascending=False)

fix_st = np.hstack(summary_school_df['School Type'])
summary_school_df['School Type'] = fix_st
fix_sb = np.hstack(summary_school_df['Total School Budget'])
summary_school_df['Total School Budget'] = fix_sb
fix_bps = np.hstack(summary_school_df['Per Student Budget'])
summary_school_df['Per Student Budget'] = fix_bps

summary_school_df['Total Students'] = summary_school_df['Total Students'].map('{:,.0f}'.format)
summary_school_df['Total School Budget'] = summary_school_df['Total School Budget'].map('$ {:,.0f}'.format)
summary_school_df['Per Student Budget'] = summary_school_df['Per Student Budget'].map('$ {:,.0f}'.format)
summary_school_df['Average Math Score'] = summary_school_df['Average Math Score'].map('{:,.2f}'.format)
summary_school_df['Average Reading Score'] = summary_school_df['Average Reading Score'].map('{:,.2f}'.format)
summary_school_df['% Passing Math'] = summary_school_df['% Passing Math'].map('{:,.2%}'.format)
summary_school_df['% Passing Reading'] = summary_school_df['% Passing Reading'].map('{:,.2%}'.format)
summary_school_df['% Overall Passing Rate'] = summary_school_df['% Overall Passing Rate'].map('{:,.2%}'.format)
summary_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Wilson High School,Charter,"2,283","$ 1,319,574",$ 578,83.27,83.99,90.93%,93.25%,92.09%
Pena High School,Charter,962,"$ 585,858",$ 609,83.84,84.04,91.68%,92.20%,91.94%
Wright High School,Charter,"1,800","$ 1,049,400",$ 583,83.68,83.95,90.28%,93.44%,91.86%
Cabrera High School,Charter,"1,858","$ 1,081,356",$ 582,83.06,83.98,89.56%,93.86%,91.71%
Holden High School,Charter,427,"$ 248,087",$ 581,83.80,83.81,90.63%,92.74%,91.69%


In [4]:
from pandas.api.types import CategoricalDtype
school_data_complete["grade"] = school_data_complete['grade'].astype(CategoricalDtype(["9th", "10th","11th","12th"]))
math_scores_grade = round(school_data_complete.pivot_table(index="school_name",columns="grade", values="math_score"),2)
math_scores_grade.index.name = None
math_scores_grade.columns = ["9th","10th","11th","12th"]
math_scores_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


In [5]:
from pandas.api.types import CategoricalDtype
school_data_complete["grade"] = school_data_complete['grade'].astype(CategoricalDtype(["9th", "10th","11th","12th"]))
reading_scores_grade = round(school_data_complete.pivot_table(index="school_name",columns="grade", values="reading_score"),2)
reading_scores_grade.index.name = None
reading_scores_grade.columns = ["9th","10th","11th","12th"]
reading_scores_grade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


In [6]:
summary_school_df2 = pd.DataFrame(data=s_type)
summary_school_df2 = summary_school_df2.join([tot_students,tot_budget,per_sb,avg_math_school,avg_reading_school,
                        result_df['% Passing Math'],result_df['% Passing Reading'], 
                        result_df['% Overall Passing Rate']])
summary_school_df2 = summary_school_df2.rename(columns={'school_name':'','Student ID': 'Total Students',
                                                     0: 'Per Student Budget',
                                                     'math_score': 'Average Math Score',
                                                     'reading_score': 'Average Reading Score','budget':'Total School Budget','type':'School Type'})

In [7]:
bins = [0, 585, 615, 645, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]
scoreByBudget = summary_school_df2[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].groupby(pd.cut(summary_school_df2["Per Student Budget"], bins=bins, labels=group_names )).mean()

scoreByBudget['Average Math Score'] = scoreByBudget['Average Math Score'].map('{:,.2f}'.format)
scoreByBudget['Average Reading Score'] = scoreByBudget['Average Reading Score'].map('{:,.2f}'.format)
scoreByBudget['% Passing Math'] = scoreByBudget['% Passing Math'].map('{:,.0%}'.format)
scoreByBudget['% Passing Reading'] = scoreByBudget['% Passing Reading'].map('{:,.0%}'.format)
scoreByBudget['% Overall Passing Rate'] = scoreByBudget['% Overall Passing Rate'].map('{:,.0%}'.format)

scoreByBudget.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
0 to 585,83.46,83.93,90%,93%,92%
585 to 615,83.60,83.89,91%,92%,92%
615 to 645,79.08,81.89,73%,83%,78%
645 to 675,77.00,81.03,64%,78%,71%


In [8]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

scores_size = summary_school_df2.loc[:,['Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Add a new columns named School Size and binning based off total students
scores_size['School Size']= pd.cut(summary_school_df2['Total Students'],size_bins,labels=group_names)
# Create a group based off of the bins
scores_size = scores_size.groupby('School Size').mean()

scores_size['Average Math Score'] = scores_size['Average Math Score'].map('{:,.2f}'.format)
scores_size['Average Reading Score'] = scores_size['Average Reading Score'].map('{:,.2f}'.format)
scores_size['% Passing Math'] = scores_size['% Passing Math'].map('{:,.0%}'.format)
scores_size['% Passing Reading'] = scores_size['% Passing Reading'].map('{:,.0%}'.format)
scores_size['% Overall Passing Rate'] = scores_size['% Overall Passing Rate'].map('{:,.0%}'.format)

scores_size.head()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,91%,92%,92%
Medium (1000-2000),83.37,83.86,90%,93%,92%
Large (2000-5000),77.75,81.34,68%,80%,74%


In [9]:
by_type = school_data_complete.groupby("type")

avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()

scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})
    
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "Type of School"


scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%
